<a href="https://colab.research.google.com/github/Jieeee412/Lab10/blob/main/Compval_Calculator_Lending_Fees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import re

with open("Lending fees 7.json", "r") as file:
    raw_data_7 = json.load(file)

with open("Lending fees 8.json", "r") as file:
    raw_data_8 = json.load(file)

In [2]:
def extract_json_array(raw_text):
    """
    Extracts and parses a JSON array from a raw string wrapped in \boxed{} and/or ```json markdown.
    Returns a Python list.
    """
    if not raw_text:
        return []

    # Remove \boxed{...} wrapper
    boxed_match = re.search(r'\\boxed\s*{(.*)}\s*}$', raw_text.strip(), flags=re.DOTALL)
    content = boxed_match.group(1) if boxed_match else raw_text

    # Remove markdown markers like ```json
    content = re.sub(r'```json|```', '', content, flags=re.IGNORECASE).strip()

    # Extract valid JSON array using [ ... ]
    start = content.find('[')
    end = content.rfind(']') + 1
    if start == -1 or end == -1:
        print("⚠️ Could not find JSON array brackets.")
        return []

    json_array_str = content[start:end]

    # Parse it
    try:
        return json.loads(json_array_str)
    except json.JSONDecodeError as e:
        print(" JSON parsing failed:", e)
        return []

# Extract and combine
raw_text_1 = raw_data_7.get("page_1", "")
raw_text_2 = raw_data_8.get("page_1", "")

fees_1 = extract_json_array(raw_text_1)
fees_2 = extract_json_array(raw_text_2)

Loan_fees = fees_1 + fees_2


In [3]:
with open("Loan_fees.json", "w") as f:
    json.dump(Loan_fees, f, indent=4)

In [4]:
# Load file
with open("/content/Loan_fees.json", "r") as f:
    raw_data = json.load(f)

# Normalize to flat fee list
normalized_fees = []

for item in raw_data:
    if "fees" in item:  # structured format
        product_type = item.get("product_type", "Unknown")
        for fee in item["fees"]:
            fee["product_type"] = product_type
            normalized_fees.append(fee)
    else:
        normalized_fees.append(item)  # already flat

print(f" Total normalized fees: {len(normalized_fees)}")

 Total normalized fees: 13


In [5]:
class FeeCalculator:
    def __init__(self, fee_list):
        self.fees = fee_list

    def list_product_types(self):
        return sorted(set(fee.get('product_type', 'Unknown') for fee in self.fees))

    def list_fee_names(self):
        return sorted(set(fee.get('fee_name', 'Unnamed') for fee in self.fees))

    def get_fees_by_product_type(self, product_type):
        return [fee for fee in self.fees if fee.get('product_type') == product_type]

    def get_fee_by_name(self, fee_name):
        return [fee for fee in self.fees if fee.get('fee_name') == fee_name]

    def calculate_total_fee(self, product_type=None):
        total = 0
        for fee in self.fees:
            if product_type is None or fee.get('product_type') == product_type:
                try:
                    amt = float(fee.get("amount"))
                    total += amt
                except (TypeError, ValueError):
                    pass  # Skip 'null' or 'variable'
        return total

    def show_fee_summary(self, product_type=None):
        selected = self.get_fees_by_product_type(product_type) if product_type else self.fees
        for fee in selected:
            product = fee.get('product_type', 'Unknown')
            name = fee.get('fee_name', 'Unnamed')
            amount = fee.get('amount', 'N/A')
            unit = fee.get('unit', 'NZD')
            timing = fee.get('charge_timing', 'N/A')
            conditions = fee.get('conditions', '')

            print(f" {product} → {name}: {amount} {unit} ({timing})")
            if conditions:
                print(f" {conditions}")
            print()

In [6]:
calc = FeeCalculator(normalized_fees)

# List all product types
print(" Product Types:", calc.list_product_types())

# Summary for one product
calc.show_fee_summary("Home Loan")

# Total of numeric-only fees
print(" Total for Home Loan:", calc.calculate_total_fee("Home Loan"))

 Product Types: ['ANZ Flexible Home Loan', 'Home Loan', 'Personal Loan', 'secured loans']
 Home Loan → Application fee (owner-occupied): None NZD (once-off)
 no charge

 Home Loan → Application fee (all other lending): None NZD (once-off)
 no charge

 Home Loan → Loan Top Up fees: None NZD (once-off)
 no charge

 Home Loan → Tideover application/extension fee: None NZD (once-off)
 no charge

 Home Loan → Early Repayment Recovery: variable NZD (event-based)
 depends on loss at the time of repayment

 Home Loan → Reserving a fixed interest rate: None NZD (once-off)
 no charge

 Home Loan → Non-utilisation fee (home lending): 25 NZD (once-off)
 applies if the reserved fixed interest rate is not utilized

 Home Loan → Non-utilisation recovery (business lending): variable NZD (event-based)
 depends on loss at the time of cancellation or restructure

 Total for Home Loan: 25.0


In [13]:
fee_lookup = {}

for fee in normalized_fees:
    product = fee.get("product_type", "").strip()
    fee_name = fee.get("fee_name", "").strip()
    if product and fee_name:
        fee_lookup[(product, fee_name)] = fee


In [14]:
print("Keys in fee_lookup:")
for key in fee_lookup.keys():
    print(key)

Keys in fee_lookup:
('Home Loan', 'Application fee (owner-occupied)')
('Home Loan', 'Application fee (all other lending)')
('Home Loan', 'Loan Top Up fees')
('Home Loan', 'Tideover application/extension fee')
('Home Loan', 'Early Repayment Recovery')
('Home Loan', 'Reserving a fixed interest rate')
('Home Loan', 'Non-utilisation fee (home lending)')
('Home Loan', 'Non-utilisation recovery (business lending)')
('ANZ Flexible Home Loan', 'Monthly account fee')
('ANZ Flexible Home Loan', 'All transaction fees (including manual transactions)')
('ANZ Flexible Home Loan', 'Unarranged overdraft fee')
('Personal Loan', 'Application fee (For new and top-up applications)')
('secured loans', 'Discharge or Execution fee')


In [7]:
import pandas as pd

home_loan_df = pd.read_csv("Home_Loan_transactions.csv")
personal_loan_df = pd.read_csv("Personal_Loans_transaction.csv")

# Append the second DataFrame to the first
combined_loan_df = pd.concat([home_loan_df, personal_loan_df], ignore_index=True)

combined_loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36816 entries, 0 to 36815
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction_ID      36816 non-null  object 
 1   Account_ID          36816 non-null  object 
 2   Account_Type        36816 non-null  object 
 3   Product_Type        36816 non-null  object 
 4   Transaction_Type    36816 non-null  object 
 5   Transaction_Amount  36816 non-null  float64
 6   Transaction_Date    36816 non-null  object 
 7   Principal           36816 non-null  object 
 8   Interest            36816 non-null  object 
 9   Description         36816 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.8+ MB


In [8]:
combined_loan_df.Product_Type.unique()

array(['Home loans_2 years', 'Home loans_4 years', 'Home loans_3 years',
       'Home loans_18 months', 'Home loans_1 year', 'Home loans_5 years',
       'Home loans_6 months', 'Home loans_Floating Rate',
       'Home loans_Flexible Rate', 'Person Loans'], dtype=object)

In [9]:
combined_loan_df = combined_loan_df.replace('Home loans_Flexible Rate', 'ANZ Flexible Home Loan')

In [10]:
combined_loan_df = combined_loan_df.replace(['Home loans_2 years', 'Home loans_4 years', 'Home loans_3 years',
       'Home loans_18 months', 'Home loans_1 year', 'Home loans_5 years',
       'Home loans_6 months', 'Home loans_Floating Rate'], 'Home Loan')

In [11]:
combined_loan_df = combined_loan_df.replace('Person Loans', 'Person Loan')

In [12]:
combined_loan_df.Product_Type.unique()

array(['Home Loan', 'ANZ Flexible Home Loan', 'Person Loan'], dtype=object)

In [15]:
from difflib import get_close_matches

transaction_types = list(combined_loan_df["Transaction_Type"].unique())
transaction_to_fee_name = {}

for fee in normalized_fees:
    product = fee.get("product_type", "").strip()
    fee_name = fee.get("fee_name", "").strip()

    # Try to find closest matching Transaction_Type
    match = get_close_matches(fee_name, transaction_types, n=1, cutoff=0.6)
    if match:
        transaction_type = match[0]
        transaction_to_fee_name[(product, transaction_type)] = fee_name


In [17]:
print("Auto-generated mapping:")
for k, v in transaction_to_fee_name.items():
    print(f"{k} → {v}")


Auto-generated mapping:
('Home Loan', 'Early Repayment Fee') → Early Repayment Recovery
('ANZ Flexible Home Loan', 'Monthly Account Fee') → Monthly account fee


In [18]:
def apply_fee_info(row):
    product_type = row.get("Product_Type", "").strip()
    trans_type = row.get("Transaction_Type", "").strip()

    fee_name = transaction_to_fee_name.get((product_type, trans_type))
    if not fee_name:
        return None

    fee_key = (product_type, fee_name)
    fee_info = fee_lookup.get(fee_key)

    if fee_info:
        return fee_info.get("amount")
    return None


In [19]:
combined_loan_df["Fee_Amount"] = combined_loan_df.apply(apply_fee_info, axis=1)


In [20]:
matched = combined_loan_df.dropna(subset=["Fee_Amount"])

fee_summary = matched.groupby("Product_Type")["Fee_Amount"].sum().reset_index()
print(fee_summary)


             Product_Type  Fee_Amount
0  ANZ Flexible Home Loan    103500.0
